In [13]:
import os
import sys

current_dir = os.getcwd()

# Walk up the directory tree until we find 'src'
path = current_dir
src_path = None

while True:
    if os.path.basename(path) == "src":
        src_path = path
        break
    parent = os.path.dirname(path)
    if parent == path:  # reached filesystem root
        break
    path = parent

# Add src to sys.path if found
if src_path and src_path not in sys.path:
    sys.path.insert(0, src_path)

# Import AutoRec model and data utilities
from utils.model import AutoRec
from utils.autorecdata import AutoRecData
from utils.preprocessor import PreProcessor
from helpers.data_downloader import download_ml1m_dataset
# Import external libraries
import torch
import torch.utils.data as data
import torch.nn as nn
import torch.optim as optim
import pandas as pd
import numpy as np
import os
from tqdm.auto import tqdm
print("=" * 70)
print("Imports Successful")
print("=" * 70)

Imports Successful


In [14]:
data_dir = os.path.join(os.path.dirname(os.getcwd()), '..', 'data')
data_path = os.path.join(data_dir,'ml-1m', 'ratings.dat')
print(data_path)
print(data_dir)
# Check if file exists
if not os.path.exists(data_path):
    download_ml1m_dataset(data_dir=data_dir)

def load_ml_1m_data(data_path = data_path) -> pd.DataFrame:  
    print("=" * 70)
    print("Loading MovieLens 1M Dataset")
    print("=" * 70)
    print(f"Data path: {data_path}")
    return pd.read_csv(
        data_path,
        sep='::',
        header=None,
        names=['user_id', 'item_id', 'rating', 'timestamp'],
        engine='python',  # Explicitly use python engine to avoid warning
        dtype={
            'user_id': np.int32,
            'item_id': np.int32,
            'rating': np.float32,
            'timestamp': np.int32
        }
    )


# Load ratings data with proper engine to avoid warnings
print("\nLoading ratings data...")

ratings_df = load_ml_1m_data()

print(f"✓ Successfully loaded {len(ratings_df):,} ratings")
print("=" * 70)

/Users/abbas/Documents/Codes/thesis/recommender/src/../data/ml-1m/ratings.dat
/Users/abbas/Documents/Codes/thesis/recommender/src/../data

Loading ratings data...
Loading MovieLens 1M Dataset
Data path: /Users/abbas/Documents/Codes/thesis/recommender/src/../data/ml-1m/ratings.dat
✓ Successfully loaded 1,000,209 ratings


In [15]:
# Create PreProcessor instance
preprocessor = PreProcessor()

# Preprocess the data: split into train/test and create rating matrices
print("\nSplitting data into train/test sets and creating rating matrices...")
print("=" * 70)
train_mat, test_mat, num_users, num_items = preprocessor.preprocess_ml1m_data(
    ratings_df, 
    test_size=0.2,  # 20% for testing
    random_state=42  # For reproducibility
)

print(f"✓ Data preprocessing complete!")
print("=" * 70)


Splitting data into train/test sets and creating rating matrices...
✓ Data preprocessing complete!


In [16]:
train_set = AutoRecData(data=train_mat)
test_set = AutoRecData(data=test_mat)

In [17]:
train_loader = data.DataLoader(
    dataset=train_set,
    batch_size=256,
    shuffle=True,
    num_workers=0,
)
print('Train loader created')
print("=" * 70)
test_loader = data.DataLoader(
    dataset=test_set, batch_size=len(test_set), shuffle=False, num_workers=0
)
print('Test loader created')
print("=" * 70)

Train loader created
Test loader created


In [18]:
# Set device (CPU or CUDA if available)
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Using device: {device}")

model = AutoRec(
    num_users=num_users,
    num_items=num_items,
    num_hidden_units=500,
).to(device)

Using device: cpu


In [19]:
loss_f = nn.MSELoss()
optimizer = optim.Adam(
    model.parameters(),
    lr=0.001,
    weight_decay=0.01,
)

best_epoch, best_rmse = 0, np.inf
best_hr_at_10, best_ndcg_at_10 = 0.0, 0.0


print(model)

AutoRec(
  (encoder): Sequential(
    (0): Linear(in_features=3706, out_features=500, bias=True)
    (1): Sigmoid()
  )
  (decoder): Sequential(
    (0): Linear(in_features=500, out_features=3706, bias=True)
  )
)


In [20]:
from utils.helper import get_metrics, get_ranking_metrics
for epoch in tqdm(range(40)):
    model.train()
    for input_vec in train_loader:
        input_mask = (input_vec > 0).to(device)
        input_vec = input_vec.float().to(device)

        model.zero_grad()
        reconstruction = model(input_vec)
        loss = loss_f(reconstruction * input_mask, input_vec * input_mask)
        loss.backward()
        optimizer.step()

    model.eval()
    rmse = get_metrics(model=model, train_set=train_set, test_set=test_set, device=device)
    hr_at_10, ndcg_at_10 = get_ranking_metrics(
        model=model, 
        train_set=train_set, 
        test_set=test_set, 
        top_k=10, 
        device=device
    )

    print(f"[Epoch {epoch}]:: RMSE: {rmse:.6f}, HR@10: {hr_at_10:.6f}, NDCG@10: {ndcg_at_10:.6f}")

    print("=" * 70)

    if rmse < best_rmse:
        best_rmse, best_epoch = rmse, epoch
        best_hr_at_10, best_ndcg_at_10 = hr_at_10, ndcg_at_10
        # Save best model
        print(f"Saving best model at epoch {epoch}")
        print(f"Best epoch: {best_epoch}, best_rmse: {best_rmse:.6f}")
        print("=" * 70)
        print(f"HR@10: {hr_at_10:.6f}, NDCG@10: {ndcg_at_10:.6f}")

    print(f"Done. Best epoch {best_epoch}, best_rmse: {best_rmse:.6f}.")
    print(epoch)
    print("=" * 70)


  2%|▎         | 1/40 [00:01<00:55,  1.41s/it]

[Epoch 0]:: RMSE: 2.466145, HR@10: 0.653031, NDCG@10: 0.190714
Saving best model at epoch 0
Best epoch: 0, best_rmse: 2.466145
HR@10: 0.653031, NDCG@10: 0.190714
Done. Best epoch 0, best_rmse: 2.466145.
0


  5%|▌         | 2/40 [00:02<00:50,  1.34s/it]

[Epoch 1]:: RMSE: 2.284595, HR@10: 0.655515, NDCG@10: 0.187925
Saving best model at epoch 1
Best epoch: 1, best_rmse: 2.284595
HR@10: 0.655515, NDCG@10: 0.187925
Done. Best epoch 1, best_rmse: 2.284595.
1


  8%|▊         | 3/40 [00:03<00:48,  1.31s/it]

...

[Epoch 37]:: RMSE: 2.281683, HR@10: 0.653196, NDCG@10: 0.180621
Done. Best epoch 30, best_rmse: 2.223268.
37


 98%|█████████▊| 39/40 [00:53<00:01,  1.53s/it]

[Epoch 38]:: RMSE: 2.246792, HR@10: 0.645412, NDCG@10: 0.178022
Done. Best epoch 30, best_rmse: 2.223268.
38


100%|██████████| 40/40 [00:54<00:00,  1.36s/it]

[Epoch 39]:: RMSE: 2.275295, HR@10: 0.653859, NDCG@10: 0.184137
Done. Best epoch 30, best_rmse: 2.223268.
39


In [ ]:
# Example: Load saved model for inference
# Uncomment and run this cell to load a saved model

# from utils.model import AutoRec
# import torch
# 
# # Load checkpoint
# model_dir = os.path.join(os.path.dirname(os.getcwd()), '..', 'models')
# checkpoint_path = os.path.join(model_dir, 'AutoRec.pth')  # or 'AutoRec-best.pth' for best model
# 
# checkpoint = torch.load(checkpoint_path, map_location='cpu')
# 
# # Recreate model with saved parameters
# loaded_model = AutoRec(
#     num_users=checkpoint['num_users'],
#     num_items=checkpoint['num_items'],
#     num_hidden_units=checkpoint['num_hidden_units']
# )
# 
# # Load model weights
# loaded_model.load_state_dict(checkpoint['model_state_dict'])
# loaded_model.eval()
# 
# print(f"Model loaded from: {checkpoint_path}")
# print(f"Epoch: {checkpoint['epoch']}")
# print(f"RMSE: {checkpoint['rmse']:.6f}")
# print(f"HR@10: {checkpoint['hr_at_10']:.6f}")
# print(f"NDCG@10: {checkpoint['ndcg_at_10']:.6f}")
# 
# # Now you can use loaded_model for inference
